In [ ]:
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, Dense, Flatten, BatchNormalization
import importlib
import tensorflow as tf

import sys
sys.path.append("../attacks/")
sys.path.append("../utils/")

import utils
from data_manager import load_data

# Black Box Attacks
import random_noise
import EvoStrategy
import SimbaWrapper

Only run train below if model is not available as .h5 file (in general, a baseline model should be available in 'basic_cnn_mnist.h5' to save time)

In [ ]:
# batch_size = 60 
# nb_epoch = 2
# input_shape = (28,28,1)
# model_ = Sequential()
# model_.add(Conv2D(100,kernel_size=(3, 3), activation='relu',padding='same', input_shape=input_shape))
# model_.add(MaxPooling2D(pool_size=(2, 2)))
# #model_.add(BatchNormalization())
# #model_.add(Dropout(0.2))

# model_.add(Conv2D(75, kernel_size=(3, 3),activation='relu',padding='same') )
# model_.add(MaxPooling2D(pool_size=(2, 2)))
# #model_.add(BatchNormalization())
# #model_.add(Dropout(0.2))

# model_.add(Dropout(0.25))
# model_.add(Flatten())
# model_.add(Dense(64, activation='relu'))
# model_.add(Dropout(0.5))
# model_.add(Dense(10, activation='softmax'))

# model_.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer='adam',
#               metrics=['accuracy'])


In [ ]:
(X_train, y_train), (X_test, y_test) = load_data("mnist")

In [ ]:
# history=model_.fit(X_train, y_train,
#           batch_size=batch_size,
#           epochs=nb_epoch,
#           verbose=1,
#           validation_data=(X_test, y_test))
# score = model_.evaluate(X_test, y_test, verbose=0)

In [ ]:
# model_.save("basic_cnn_mnist.h5")

In [ ]:
model_ = tf.keras.models.load_model('../models/basic_cnn_mnist.h5')

In [ ]:
sample_size_test_mnist = 100
sample_x_test = np.copy(X_test[:sample_size_test_mnist])
sample_y_test = np.copy(y_test[:sample_size_test_mnist])

In [ ]:
sample_size_train_mnist = 100
sample_x_train = np.copy(X_train[:sample_size_train_mnist])
sample_y_train = np.copy(y_train[:sample_size_train_mnist])

<h3>Random noise expriments</h3>

In [ ]:
importlib.reload(random_noise)

agreements_cell_test, saved_noisy_imgs_cell_test, number_queries = random_noise.check_noise_robustness_multiple_rounds(
    model_,                  # model to attack
    sample_x_test,           # sample images to attack
    sample_y_test,           # correct labels of the sample images
    20,                      # max number of steps to run the attack for
    noise_type="cells",      # 'cells' by default, it is gaussian noise added to a random set of pixels
    args={"cell_count":28}   # 'cell_count' is how many pixels to randomly perturb at maximum; so l0 distance <= cell_count value
)

In [ ]:
# number_queries is a dict containing pairs img_id: number_steps_to_perturb. 0 means it was already classified wrong
plt.hist(number_queries.values())

<h3>Evo strategy experiments</h3>

In [ ]:
importlib.reload(EvoStrategy)

perturbed_images = 0
adv_evo_strategy = {}
for index in range(0,100):
    print()
    print(index)
    img = sample_x_test[index]
    label = np.argmax(sample_y_test[index])
    adv_evo_strategy[index] = EvoStrategy.AdversarialPerturbationEvoStraegy(model=model_,img=img,label=label,generation_size=25)
    no_steps = adv_evo_strategy[index].run_adversarial_attack()
    if adv_evo_strategy[index].stop_criterion() and no_steps > 0:
        perturbed_images += 1

In [ ]:
count_q = 0
sum_q = 0
for index in range(0,100):
    if(adv_evo_strategy[index].queries > 1):
        count_q +=1
        sum_q += adv_evo_strategy[index].queries
        
print("Average queries for EVO algo:", sum_q/count_q)
print("Total count of perturbed images (classified correctly initially):", perturbed_images)

<h3>SimBA pixel experiments</h3>

In [ ]:
importlib.reload(simba_pixel)
simba_wrapper = simba_pixel.SimbaWrapper(model_, sample_x_test, sample_y_test, 64, 2000)

In [ ]:
simba_wrapper.run_simba()

In [ ]:
simba_wrapper.perturbed

In [ ]:
# SOTA architecture for ImageNet, run this for a chunk of data (ResNet-50, etc.) - pe imaginilie de test preferabil, aplicam atacurile
# cand l0 a depasit norma, ma opresc. numar count(queries)

<h3>CMA ES experiments below, neglect them for the moment</h3>

In [ ]:
# import cma

In [ ]:
# def get_function_for_label_and_model(label, model):
#     def evaluate_model(flat_image):
#         image = np.reshape(flat_image, (28,28,1))
#         prediction = model.predict([image])[0][label]
#         return prediction
#     return evaluate_model

In [ ]:
# index = 5
# img = sample_x_test[index]
# label = np.argmax(sample_y_test[index])

In [ ]:
# plt.imshow(np.reshape(img, (28,28)))

In [ ]:
# opt_funct = get_function_for_label_and_model(label, model_all_data)

In [ ]:
# start_point = np.reshape(img, (-1))

In [ ]:
# es = cma.CMAEvolutionStrategy(start_point, 10)

In [ ]:
# while not es.stop():
#     solutions = es.ask()
#     fitnesses = [opt_funct(x) for x in solutions]
#     index_best = np.argmin(fitnesses)
#     img = np.reshape(solutions[index_best], (28,28))
#     print(min(fitnesses))
#     plt.imshow(img)
#     plt.show()
#     es.tell(solutions, fitnesses)
#     es.logger.add()  # write data to disc to be plotted
# #     es.disp()

# # es.result_pretty()
# # cma.plot()  # shortcut for es.logger.plot()